## numpy.linspace的矢量化
**30 January 2020 by MiniUFO**

---
[TOC]

---

### 1. 问题
最近想用[python](https://www.python.org/)的[xarray](http://xarray.pydata.org/en/stable/)包来做等值线分析，中间发现一个问题，就是numpy的[```linspace(start, stop, num=50)```](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html)函数尽然不支持矢量化的参数。例如当参数start和stop都是数值时，一切都很完美：

In [25]:
import numpy as np
import numexpr as ne

start = 0
stop = 1
N   = 5

result = np.linspace(start, stop, N)
result

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

但是当参数是1D数组的时候（暂时不考虑高维数组），```linspace```就智障了：

In [26]:
start = np.array([0, 1, 2])
stop = np.array([3, 4, 5])
N   = 5

result = np.linspace(start, stop, N)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

我希望结果是相当于调用三次```linspace```，分别是```linspace(0, 3, N)```，```linspace(1, 4, N)```，```linspace(2, 5, N)```，然后再合并(concat)成一个numpy array。结果居然出错。以我现有的智商水平无法理解它的错误提示。

### 2. 解决方法
搜索了一圈，果然很多人在**stack overflow**上提问了。下面整理了好几种解决方案（写这篇文章的时候```numpy```已经更新到```1.18.1```，```linspace```已经支持矢量化了，但是家里笔记本上还是```1.15.4```，说明在线更新太@!5%#...的重要了）：

In [27]:
def solution1(start, stop, N, endpoint=True):
    if endpoint:
        divisor = N-1
    else:
        divisor = N
        
    s0 = start[:,None]
    s1 = stop [:,None]
    r  = np.arange(N)
    
    return ne.evaluate('((1.0/divisor) * (s1 - s0))*r + s0')
    
def solution2(start, stop, N, endpoint=True):
    if endpoint:
        divisor = N-1
    else:
        divisor = N
        
    steps = (1.0 / divisor) * (stop - start)
    return steps[:,None] * np.arange(N) + start[:,None]

def solution3(start, stop, N):
    return (np.linspace(0,1,N)[:,None] * (stop - start) + start).T

def solution4(start, stop, N):
    return np.array([np.linspace(i, j, N) for i,j in zip(start,stop)])

r1 = solution1(start, stop, N)
r2 = solution2(start, stop, N)
r3 = solution3(start, stop, N)
r4 = solution4(start, stop, N)

print('r1:\n' + str(r1))
print('r2:\n' + str(r2))
print('r3:\n' + str(r3))
print('r4:\n' + str(r4))

r1:
[[0.   0.75 1.5  2.25 3.  ]
 [1.   1.75 2.5  3.25 4.  ]
 [2.   2.75 3.5  4.25 5.  ]]
r2:
[[0.   0.75 1.5  2.25 3.  ]
 [1.   1.75 2.5  3.25 4.  ]
 [2.   2.75 3.5  4.25 5.  ]]
r3:
[[0.   0.75 1.5  2.25 3.  ]
 [1.   1.75 2.5  3.25 4.  ]
 [2.   2.75 3.5  4.25 5.  ]]
r4:
[[0.   0.75 1.5  2.25 3.  ]
 [1.   1.75 2.5  3.25 4.  ]
 [2.   2.75 3.5  4.25 5.  ]]


结果完全一样，再看看性能：

In [28]:
%timeit solution1(start, stop, 10000)
%timeit solution2(start, stop, 10000)
%timeit solution3(start, stop, 10000)
%timeit solution4(start, stop, 10000)

184 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
109 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
362 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
144 µs ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


结果显示第二种方法最快，ok，就用这个函数来做矢量化的```linspace```。

#### References:
https://stackoverflow.com/questions/40624409/vectorized-numpy-linspace-for-multiple-start-and-stop-values/40624614#40624614
https://stackoverflow.com/questions/16887148/python-linspace-limits-from-two-arrays/16887295#16887295